## Qwen2.5-0.5B-Instruct 模型下载部署调用标准流程
1. 下载必要的库：modelscope transformers torch accelerate   这里由于国内环境，我用modelscope下载
2. 使用的包为：
      - snapshot_download 来下载模型
      - AuToModelForCausalLM，AutoTokenizer来加载模型    
3. 确定模型的名称：Qwen/Qwen2.5-0.5B-Instruct, 下载模型保存到路径model_dir中
4. 加载模型
5. 准备数据：prompt 和 messages
6. 将输入的数据进行apply_chat_template处理
7. 将处理好的数据转化成tokens，变成model_inputs
8. 生成回答的tokens，为generated_ids，并且只保留LLM回答的部分
9. 将第八步的结果进行decode解码
10. 输出最终的文本回答

In [2]:
from modelscope import snapshot_download
from modelscope import AutoModelForCausalLM,AutoTokenizer

In [3]:
model_id = "Qwen/Qwen2.5-0.5B-Instruct"

In [4]:
model_dir = snapshot_download(model_id)

2025-12-30 14:01:09,220 - modelscope - INFO - Got 1 files, start to download ...


Processing 1 items:   0%|          | 0.00/1.00 [00:00<?, ?it/s]

2025-12-30 14:05:35,883 - modelscope - INFO - Download model 'Qwen/Qwen2.5-0.5B-Instruct' successfully.
2025-12-30 14:05:35,899 - modelscope - INFO - Creating symbolic link [C:\Users\User\.cache\modelscope\hub\models\Qwen\Qwen2.5-0.5B-Instruct].
2025-12-30 14:05:35,899 - modelscope - WARNING - Failed to create symbolic link C:\Users\User\.cache\modelscope\hub\models\Qwen\Qwen2.5-0.5B-Instruct for C:\Users\User\.cache\modelscope\hub\models\Qwen\Qwen2___5-0___5B-Instruct.


In [13]:
model_dir

'C:\\Users\\User\\.cache\\modelscope\\hub\\models\\Qwen\\Qwen2___5-0___5B-Instruct'

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_dir,trsut_remote_code = True)

In [12]:
tokenizer

Qwen2TokenizerFast(name_or_path='C:\Users\User\.cache\modelscope\hub\models\Qwen\Qwen2___5-0___5B-Instruct', vocab_size=151643, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>",

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map = "auto",
    torch_dtype = "auto",
    trust_remote_code = True
)

In [11]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [7]:
prompt = "解释一下LM和LMM的区别是什么"

In [8]:
messages = [
    {"role":"user","content":prompt}
]

In [9]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True
)

In [10]:
text

'<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n解释一下LM和LMM的区别是什么<|im_end|>\n<|im_start|>assistant\n'

In [15]:
model_inputs = tokenizer([text],return_tensors = 'pt').to(model.device)

In [16]:
model_inputs

{'input_ids': tensor([[151644,   8948,    198,   2610,    525,   1207,  16948,     11,   3465,
            553,  54364,  14817,     13,   1446,    525,    264,  10950,  17847,
             13, 151645,    198, 151644,    872,    198, 104136, 100158,  10994,
          33108,     43,   8035, 109226, 102021, 151645,    198, 151644,  77091,
            198]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [26]:
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens = 512
)

In [27]:
generated_ids

tensor([[151644,   8948,    198,   2610,    525,   1207,  16948,     11,   3465,
            553,  54364,  14817,     13,   1446,    525,    264,  10950,  17847,
             13, 151645,    198, 151644,    872,    198, 104136, 100158,  10994,
          33108,     43,   8035, 109226, 102021, 151645,    198, 151644,  77091,
            198,  10994,  58143,    444,   8035,  54851, 101441, 101970, 104949,
         104034,  39907,   3837, 104017,  18493,  54542,  99795, 102064,  86119,
          13343, 114046, 100772,  33108, 105255, 102122,   3407,     16,     13,
           3070,  10994,   9909,  13806,   4903,   7552,    334,  28311,    256,
            481,  46414,   6567,  44401,  24300,  99558, 100020, 100146,  43959,
         108704, 108530,   1773,  77557,  96050, 102182, 105395,   5373, 104934,
         101042,   5373, 111436,  72448,  49567,  88802,  15946,   3837,  10994,
            220,  36993, 104482, 104538, 108725,  99689,  57191,  99534,  72881,
           9370, 109091,   8

In [28]:
generated_ids = [
    output_ids[len(input_ids):] for input_ids,output_ids in zip(model_inputs.input_ids,generated_ids)
]

In [29]:
generated_ids

[tensor([ 10994,  58143,    444,   8035,  54851, 101441, 101970, 104949, 104034,
          39907,   3837, 104017,  18493,  54542,  99795, 102064,  86119,  13343,
         114046, 100772,  33108, 105255, 102122,   3407,     16,     13,   3070,
          10994,   9909,  13806,   4903,   7552,    334,  28311,    256,    481,
          46414,   6567,  44401,  24300,  99558, 100020, 100146,  43959, 108704,
         108530,   1773,  77557,  96050, 102182, 105395,   5373, 104934, 101042,
           5373, 111436,  72448,  49567,  88802,  15946,   3837,  10994,    220,
          36993, 104482, 104538, 108725,  99689,  57191,  99534,  72881,   9370,
         109091,   8997,    256,    481,  46414,  43589,  66017, 112452, 104210,
          98841, 104034, 109824, 104949,  36407,  43959,   9370,   1773,  99652,
         102119,  37029,  98841,  91282,   9370, 104190, 100631, 105149,   9370,
         107018,  36407, 103930,  43959,   9370, 108704,  43815,   8997,    256,
            481,  46414,  26

In [30]:
response = tokenizer.batch_decode(
    generated_ids,
    skip_special_tokens = True
)[0]

In [31]:
print(response)

LM 和 LMM 是两种不同的模型训练方法，它们在处理自然语言问题时有不同的特点和适用场景。

1. **LM（Language Model）**：
   - LM 模型主要关注的是生成文本的任务。例如，在机器翻译、情感分析、问答系统等任务中，LM 会尝试预测下一个词或短语的含义。
   - LM 的输出通常是基于预训练的语言模型来生成的。它通常使用预定义的规则或者特定的算法来决定生成的文本内容。
   - LM 可以用于生成文章、评论、报告等各种类型的文本。

2. **LMM（Large Language Model）**：
   - LMM 相比于 LM 更加强大，它可以学习到更丰富的知识，并且可以进行更复杂的任务。LMM 不仅能够生成文本，还可以理解上下文信息，甚至理解和预测其他类型的自然语言问题。
   - LMM 通过大量的数据训练来提高其对不同领域知识的理解能力。LMM 还具有更强的学习能力和推理能力，能够在面对复杂问题时表现出色。
   - LMM 可以应用于各种应用领域，如图像识别、语音识别、自然语言理解和对话系统等。

总结来说，LM 主要用于生成文本，而 LMM 则是大模型技术的一种，可以进行更加全面和复杂的任务，包括但不限于理解、推理、学习等。两者都是自然语言处理领域的核心技术，各有千秋。
